In [2]:
#Required Imports lo load pyspark
import pyspark
from pyspark.sql import SparkSession

In [3]:
#Creating Spark Session
spark=SparkSession.builder.appName("BigData").getOrCreate()

In [4]:
spark

In [5]:

#Below code is used to seperate the file names based on the year, month and, map them to seperate lists in a dictionary

import os

#Defining the folder path and the year variables required to store the data.
folder_path = r"C:\Users\kapil\Downloads\Compressed\COVID-19-master\COVID-19-master\csse_covid_19_data\csse_covid_19_daily_reports_us"
date_strings = os.listdir(folder_path) #accessing the files in the above folder as a ist
year2021=[]
year2022=[]
year2023=[]
years=["2021","2022","2023"]
yearsData={}

#Looping through the files to segregate into different lists  based on years and their respective  months
for i in range(12):
    year2021.append([])
    year2022.append([])
    year2023.append([])
for year in years:
    for i in date_strings:
        n=i.split(".")[0]
        if(n.endswith(year)):
            x=int(n.split("-")[0])
            if(year=="2021"):
                year2021[x-1].append(i)
            elif(year=="2022"):
                year2022[x-1].append(i)
            elif(year=="2023"):
                year2023[x-1].append(i)


yearsData["2021"]=year2021
yearsData["2022"]=year2022
yearsData["2023"]=year2023
Dataframe={"2021":[],"2022":[],"2023":[]}

#Loading the data inside the individual files of a month into one dataframe
#such that we will have one dataframe for a month by combining all the csv files per month into one 
for year in yearsData:
    for m in yearsData[year]:
        month=[]
        for k in m:
            folder_path = r"C:\Users\kapil\Downloads\Compressed\COVID-19-master\COVID-19-master\csse_covid_19_data\csse_covid_19_daily_reports_us"
            folder_path+='\\'+k
            data=spark.read.option('header',"true").csv(folder_path,inferSchema=True) 
            # we only extract the State and Fatality_Ratio as the rest of the data is not significant for this usecase
            month.append(data.select("Province_State","Case_Fatality_Ratio")) 
        Dataframe[year].append(month)
    print("Successfully loaded ",year," data")        


Successfully loaded  2021  data
Successfully loaded  2022  data
Successfully loaded  2023  data


In [10]:
#This section will find the average Case_Fatality_Ratio for all Sates from all months and seperate them based on years 

from functools import reduce  
from pyspark.sql import DataFrame

for year in Dataframe:
    month_no=1
    for month in Dataframe[year]:
        state=[]
        fatality=[]
        total={"Province_State":{}}
        if(len(month)>0):
            finalresult=reduce(DataFrame.unionAll, month)#combining the data extracted for a month into single dataframe 
            data=finalresult.select("Province_State","Case_Fatality_Ratio").collect()
            for i in data:
                x=i.asDict()
                state.append(x["Province_State"])
                fatality.append(x["Case_Fatality_Ratio"])
                if(x["Province_State"] not in total["Province_State"]):
                    total["Province_State"][x["Province_State"]]=[]
                else:
                    total["Province_State"][x["Province_State"]].append(x["Case_Fatality_Ratio"])

            #Extracting the top ten States that have the highest Fatality_Ratio and printing them
            final_dict={}
            for i in total:
                for j in total[i]:
                    final_dict[j]=sum(filter(None,total[i][j]))
            result=dict(sorted(final_dict.items(),reverse=True, key=lambda item: item[1]))
            print("Top ten States with high Infection Fatality Rate for ",year,"/",month_no," are :")
            month_no+=1
            for i in list(result.items())[:10]:
                print(i[0]," : ",i[1])
            print()
            #Below function call will pass all states data for individial months to a organize and plot the Graph
            #convertDataToPlot(result) #please uncomment his line inorder to see graph for each month



Top ten States with high Infection Fatality Rate for  2021 / 1  are :
New York  :  100.41003616912006
New Jersey  :  98.82451780975222
Connecticut  :  88.80465790630551
Massachusetts  :  88.7667819930075
Grand Princess  :  87.37864077669897
District of Columbia  :  76.37331280846894
Michigan  :  75.3798562933295
Pennsylvania  :  74.77000387358375
Louisiana  :  67.68253794244089
Mississippi  :  65.60530318606335



Top ten States with high Infection Fatality Rate for  2021 / 2  are :
New Jersey  :  81.29553285971645
New York  :  80.09979724590833
Grand Princess  :  78.64077669902909
Massachusetts  :  74.85896066867205
Connecticut  :  74.40660357156351
Pennsylvania  :  69.05528244158579
Michigan  :  68.99546130314833
District of Columbia  :  67.70430196350112
Mississippi  :  60.645623333112205
Louisiana  :  60.02728553668497



Top ten States with high Infection Fatality Rate for  2021 / 3  are :
Grand Princess  :  87.37864077669897
New Jersey  :  84.79010289357647
New York  :  83.40724679088876
Massachusetts  :  82.5820546367576
Connecticut  :  79.28934667393355
Pennsylvania  :  75.11750146884229
Michigan  :  73.74811323436064
District of Columbia  :  73.31434459348189
Mississippi  :  68.83627405627479
Louisiana  :  67.87064840569523



Top ten States with high Infection Fatality Rate for  2021 / 4  are :
Grand Princess  :  84.46601941747568
Massachusetts  :  75.86058832139108
New Jersey  :  75.24303114470628
New York  :  75.04200273029178
Connecticut  :  70.81279507861446
District of Columbia  :  68.13468606776487
Pennsylvania  :  67.27036092802155
Mississippi  :  66.98360261913577
Louisiana  :  65.92296907839253
Michigan  :  60.702958338067965



Top ten States with high Infection Fatality Rate for  2021 / 5  are :
Grand Princess  :  87.37864077669897
New Jersey  :  77.05224492536315
Massachusetts  :  75.99905670904973
New York  :  75.91169275816438
Connecticut  :  71.1874444708073
District of Columbia  :  69.29150999921912
Mississippi  :  69.139061350661
Louisiana  :  67.59606084262657
Pennsylvania  :  67.13385228977884
New Mexico  :  61.932028822103504



...

Top ten States with high Infection Fatality Rate for  2022 / 11  are :
Grand Princess  :  84.46601941747568
Pennsylvania  :  41.79236492521807
Oklahoma  :  40.99517758195445
Georgia  :  40.426233513824066
Mississippi  :  40.29773287228845
Arizona  :  39.750359955567774
New Mexico  :  39.501195663837095
Michigan  :  39.478030142444396
Nevada  :  39.16380281596639
Alabama  :  38.7842350668685



Top ten States with high Infection Fatality Rate for  2022 / 12  are :
Grand Princess  :  87.37864077669897
Pennsylvania  :  43.07612655698786
Oklahoma  :  42.06007355327097
Georgia  :  41.762020077488
Mississippi  :  41.37982695159438
Michigan  :  40.97105622204739
Arizona  :  40.56815019521455
Nevada  :  40.23720805031194
New Mexico  :  40.16031607315672
Alabama  :  39.79465750630959



Top ten States with high Infection Fatality Rate for  2023 / 1  are :
Grand Princess  :  87.37864077669897
Pennsylvania  :  42.87799503574139
Oklahoma  :  41.675006291633075
Georgia  :  41.51765579231473
Michigan  :  40.95849028825107
Arizona  :  40.80500093230993
Mississippi  :  40.79939111347684
Nevada  :  40.282074993195685
New Mexico  :  40.26916823581184
Alabama  :  39.111415888509285



Top ten States with high Infection Fatality Rate for  2023 / 2  are :
Grand Princess  :  78.64077669902909
Pennsylvania  :  38.60142359269302
Oklahoma  :  37.54338866083601
Georgia  :  37.3875888715965
Michigan  :  37.15496431358914
Arizona  :  36.899041902662034
Mississippi  :  36.47193025871923
New Mexico  :  36.4334790625397
Nevada  :  36.24428873536563
Arkansas  :  34.8898320811666



Top ten States with high Infection Fatality Rate for  2023 / 3  are :
Grand Princess  :  23.300970873786408
Pennsylvania  :  11.4256598034281
Oklahoma  :  11.131079454519272
Georgia  :  11.074021636238053
Michigan  :  11.016856347887487
Arizona  :  10.84183946970784
New Mexico  :  10.80555275798487
Mississippi  :  10.796336887811767
Nevada  :  10.721182432125108
Arkansas  :  10.341143557377062



In [9]:
#This function will convert the full State names into a Short versions
#this conversion is needed because Plotting package needs this notation  inorder to plot the graph
def convertDataToPlot(state_values_full_form):
    state_mapping = {
    'New York': 'NY',   'New Jersey': 'NJ',    'Connecticut': 'CT',    'Massachusetts': 'MA',    'Grand Princess': 'GP',
    'District of Columbia': 'DC',    'Michigan': 'MI',    'Pennsylvania': 'PA',    'Louisiana': 'LA',    'Mississippi': 'MS',
    'Rhode Island': 'RI',    'Maryland': 'MD',    'Puerto Rico': 'PR',    'Ohio': 'OH',    'Illinois': 'IL',    'Indiana': 'IN',
    'New Mexico': 'NM',    'Delaware': 'DE',    'Guam': 'GU',    'Arizona': 'AZ',    'Vermont': 'VT',    'South Carolina': 'SC',
    'West Virginia': 'WV',    'Arkansas': 'AR',    'New Hampshire': 'NH',    'Northern Mariana Islands': 'MP',
    'South Dakota': 'SD',    'Florida': 'FL',    'Georgia': 'GA',    'Texas': 'TX',    'Maine': 'ME',
    'North Dakota': 'ND',    'Nevada': 'NV',    'Alabama': 'AL',    'Colorado': 'CO',
    'Iowa': 'IA',    'Washington': 'WA',    'Hawaii': 'HI',    'Minnesota': 'MN',
    'Oregon': 'OR',    'Kansas': 'KS',    'Virginia': 'VA',    'Missouri': 'MO',
    'Tennessee': 'TN',    'Montana': 'MT',    'North Carolina': 'NC',    'California': 'CA',
    'Wyoming': 'WY',    'Virgin Islands': 'VI',    'Wisconsin': 'WI',    'Idaho': 'ID',
    'Nebraska': 'NE',    'Kentucky': 'KY',    'Oklahoma': 'OK',    'Utah': 'UT',    'Alaska': 'AK',    'American Samoa': 'AS',    'Diamond Princess': 'DP'
    }

    state_values_short_form = {state_mapping[state]: value for state, value in state_values_full_form.items()}
    PlotMap(state_values_short_form)

In [7]:
#The following code will actually plot the graph for provided set of data will State names and IFR

import pandas as pd
import plotly.express as px

def PlotMap(state_values):
    # Create a DataFrame from the dictionary
    df = pd.DataFrame(list(state_values.items()), columns=['state_code', 'Infection Fatality Ratio'])

    # Load state code to name mapping for hover information
    state_codes = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2011_us_ag_exports.csv')
    state_mapping = dict(zip(state_codes['code'], state_codes['state']))

    # Map state codes to state names for hover information
    df['state'] = df['state_code'].map(state_mapping)

    fig = px.choropleth(
        df,
        locations='state_code',
        locationmode="USA-states",
        color='Infection Fatality Ratio',
        color_continuous_scale="Viridis_r",
        scope="usa",
        hover_data=['state', 'Infection Fatality Ratio'],
        title='Infection Fatality Ration by State'
    )

    fig.update_geos(fitbounds="locations", visible=False)

    fig.show()


C:\Users\kapil\AppData\Local\Temp\ipykernel_13944\3114799752.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
